In [10]:
import numpy as np
import torch
try:
    from tqdm import tqdm
except:
    pass
from torch.utils.data import Dataset
import torch.nn.functional as F
import torch
import torch.nn as nn
import copy
import json
import sys
sys.path.append("/home/jovyan/conv_meta/conv_meta")
from data.feature import *
import pandas as pd
from aibrain_common.data.dataset_builder import (ColumnSpec, DatasetBuilder)
from aibrain_common.component import tools
from aibrain_common.utils.date_convert_utils import DateConvertUtils

2022-09-28 13:22:33.152431: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [11]:
def source_emb_prepare(embed_need_prepare):
        embed_need_prepare = pd.DataFrame(embed_need_prepare)
        index_name_orig = embed_need_prepare.columns.values.tolist()
        for name_orig in index_name_orig:
            len_str = len(embed_need_prepare[embed_need_prepare[name_orig] != None][name_orig][0].split(','))
            null_fit_str = ','.join(['-1.0' for _ in range(len_str)])
            values = {name_orig:null_fit_str}
            embed_need_prepare.fillna(value=values)
            embed_need_prepare[name_orig].replace(to_replace={None:null_fit_str}, inplace=True)
            for i in range(len_str):
                embed_need_prepare[name_orig+str(i)] = embed_need_prepare[name_orig].apply(lambda x:x.split(',')[i])
            embed_need_prepare.drop(columns=name_orig)
        return embed_need_prepare

In [13]:
sparse_features = user_cate_cols + feed_cate_cols
dense_features = user_num_cols + feed_num_cols
def featuers_spec():
        """ Define feature column
        Args:
        """
        dense_features_spec = [ColumnSpec(column_name=x, dtype = 'float', is_label=False) for x in dense_features]

        sparse_features_spec = [ColumnSpec(column_name=x, dtype = 'int64', is_label=False) for x in sparse_features]

        label_feature_spec = [ColumnSpec(column_name='label_evehicle_mall_goodspicture', dtype='int64', is_label=True)]
        
        us_emb_feature_spec = [ColumnSpec(column_name=x, dtype = 'string', is_label=False) for x in us_emb]
        vs_emb_feature_spec = [ColumnSpec(column_name=x, dtype = 'string', is_label=False) for x in vs_emb]

        featuers_spec = dense_features_spec + sparse_features_spec + label_feature_spec + us_emb_feature_spec + vs_emb_feature_spec
    #     featuers_spec = dense_features_spec + sparse_features_spec + label_feature_spec 
    
        return featuers_spec


In [16]:
spec = featuers_spec()
train_dataset_builder = DatasetBuilder(input_table='turing_dev.cross_domain_recom_zuche_evehicle_sample_train',\
     partitions=f'pt={"20220916"}', column_spec=spec).to_pandas()
test_dataset_builder = DatasetBuilder(input_table='turing_dev.cross_domain_recom_zuche_evehicle_sample_test', \
    partitions=f'pt={"20220916"}', column_spec=spec).to_pandas()

2022-09-28 13:35:56,711 - aibrain_common.io.hdfs_reader - INFO - send http post
2022-09-28 13:35:57,040 - aibrain_common.io.hdfs_reader - INFO - send http post done: b'{"code":0,"msg":"\xe8\xa1\xa8\xe4\xbf\xa1\xe6\x81\xaf\xe6\x9f\xa5\xe8\xaf\xa2\xe6\x88\x90\xe5\x8a\x9f","data":{"":"serialization.null.format","top_10_item_click_5_evehicle_mall_goodspicture":"int","u_exposure_times_7d_evehicle_mall_goodspicture":"float","rentcars_ordercreate_cnt_30d":"bigint","rentcars_order_last_view_daydiff":"bigint","min_click_price_7d_evehicle_mall_goodspicture":"float","usr_pub_account_balance":"float","Database:":"turing_dev          ","usr_ebik_ride_arpu":"double","usr_pub_usual_active_city_lvl":"bigint","usr_pub_academic":"int","label_value":"int","rentcars_orderfinish_cnt_60d":"bigint","ui_click_times_7d_pv_evehicle_mall_goodspicture":"float","InputFormat:":"org.apache.hadoop.hive.ql.io.orc.OrcInputFormat","i_exposure_times_7d_pv_evehicle_mall_goodspicture":"float","rentcars_orderpay_gmv_60d":"d

In [19]:
def source_emb_prepare(embed_need_prepare):
        embed_need_prepare = pd.DataFrame(embed_need_prepare)
        index_name_orig = embed_need_prepare.columns.values.tolist()
        for name_orig in index_name_orig:
            len_str = len(embed_need_prepare[embed_need_prepare[name_orig] != None][name_orig][0].split(','))
            null_fit_str = ','.join(['-1.0' for _ in range(len_str)])
            values = {name_orig:null_fit_str}
            embed_need_prepare.fillna(value=values)
            embed_need_prepare[name_orig].replace(to_replace={None:null_fit_str}, inplace=True)
            for i in range(len_str):
                embed_need_prepare[name_orig+str(i)] = embed_need_prepare[name_orig].apply(lambda x:x.split(',')[i])
            embed_need_prepare.drop(columns=name_orig)
        return embed_need_prepare

In [21]:
train_dataset_builder_ = train_dataset_builder.sample(n=100000, replace=True, random_state=18).reset_index(drop=True)
s_usr_embed_df = train_dataset_builder_[us_emb]
s_usr_embed_df = source_emb_prepare(s_usr_embed_df)

In [30]:
s_item_embed_df = train_dataset_builder_[vs_emb]
len_str = len(s_item_embed_df['item_emb_0'][0].split(','))
null_fit_str = ','.join(['-1.0' for _ in range(len_str)])
print(s_item_embed_df)
for i in range(5):
    s_item_embed_df['item_emb_'+str(i)].replace(to_replace={None:null_fit_str}, inplace=True)
# s_item_embed_df = source_emb_prepare(s_item_embed_df)
print(s_item_embed_df)

                                              item_emb_0  \
0      -0.020345397293567657, -0.217990905046463, -0....   
1      -0.01403982937335968, -0.017681241035461426, -...   
2      0.010910830460488796, -0.08495029807090759, 0....   
3      -0.006651971489191055, 0.21700970828533173, -0...   
4      0.018911201506853104, 0.09183803200721741, -0....   
...                                                  ...   
99995  -0.006491950247436762, 0.2220931202173233, -0....   
99996  -0.019605334848165512, -0.19448134303092957, -...   
99997  -0.011727099306881428, 0.05578753352165222, -0...   
99998  0.026382293552160263, 0.005745112895965576, 0....   
99999  0.03877287730574608, 0.22572016716003418, -0.0...   

                                              item_emb_1  \
0                                                   None   
1                                                   None   
2      0.002153579145669937, 0.205512136220932, -0.03...   
3      -0.01641768217086792, -0.0932187

/usr/local/lib/python3.7/site-packages/pandas/core/generic.py:6746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [35]:
s_item_embed_df = train_dataset_builder_[vs_emb]
index_name_orig = s_item_embed_df.columns.values.tolist()
for name_orig in index_name_orig:
#     len_str = len(s_item_embed_df[s_item_embed_df[name_orig] != None][name_orig][0].split(','))
    print(s_item_embed_df[s_item_embed_df[name_orig].notna()])


                                              item_emb_0  \
0      -0.020345397293567657, -0.217990905046463, -0....   
1      -0.01403982937335968, -0.017681241035461426, -...   
2      0.010910830460488796, -0.08495029807090759, 0....   
3      -0.006651971489191055, 0.21700970828533173, -0...   
4      0.018911201506853104, 0.09183803200721741, -0....   
...                                                  ...   
99995  -0.006491950247436762, 0.2220931202173233, -0....   
99996  -0.019605334848165512, -0.19448134303092957, -...   
99997  -0.011727099306881428, 0.05578753352165222, -0...   
99998  0.026382293552160263, 0.005745112895965576, 0....   
99999  0.03877287730574608, 0.22572016716003418, -0.0...   

                                              item_emb_1  \
0                                                   None   
1                                                   None   
2      0.002153579145669937, 0.205512136220932, -0.03...   
3      -0.01641768217086792, -0.0932187

In [46]:
def source_emb_prepare(embed_need_prepare):
        embed_need_prepare = pd.DataFrame(embed_need_prepare)
        index_name_orig = embed_need_prepare.columns.values.tolist()
        for name_orig in index_name_orig:
            embed_temp = copy.deepcopy(embed_need_prepare)
            embed_temp_ = embed_temp[embed_temp[name_orig].notna()].reset_index(drop=True)
            len_str = len(embed_temp_[name_orig][0].split(','))
            null_fit_str = ','.join(['-1.0' for _ in range(len_str)])
            values = {name_orig:null_fit_str}
            embed_need_prepare.fillna(value=values)
            embed_need_prepare[name_orig].replace(to_replace={None:null_fit_str}, inplace=True)
            for i in range(len_str):
                embed_need_prepare[name_orig+str(i)] = embed_need_prepare[name_orig].apply(lambda x:x.split(',')[i])
            print('name_orig', name_orig)
            embed_need_prepare.drop(columns=name_orig, inplace=True)
        return embed_need_prepare

In [47]:
s_item_embed_df = train_dataset_builder_[vs_emb]
s_item_embed_df = source_emb_prepare(s_item_embed_df)

name_orig item_emb_0
name_orig item_emb_1
name_orig item_emb_2
name_orig item_emb_3
name_orig item_emb_4


In [48]:
print(s_item_embed_df.shape)

(100000, 320)


In [43]:
s_item_embed_df = pd.DataFrame(s_item_embed_df)
index_name_orig = s_item_embed_df.columns.values.tolist()
print(index_name_orig[0])

item_emb_0


In [53]:
def cat_embeding_prepare(feature_df):
        
        with open('/home/jovyan/conv_meta/conv_meta/data/20220916_label_encoder_dict.json', 'r',encoding='utf8') as fp:
            le_dict = json.load(fp)
        label_rows = le_dict
        label_max_value_dict = {}
        for (k, v) in label_rows.items():
            max_value = 0
            for (kk, vv) in v.items():
                if k in feature_df.columns.tolist():
                    feature_df[feature_df[k]==kk] = vv
                if vv >= max_value:  
                    max_value = vv
            label_max_value_dict[k] = max_value + 1
        for col in feature_df.columns.tolist():
            size_ =  label_max_value_dict[col]
            embedding_layer = nn.Embedding(size_, 10)
            embed_col = embedding_layer(torch.tensor(feature_df[col], dtype=torch.long)).view(feature_df.shape[0], 10)
            feature_df.drop(columns=[col])
            feature_df = pd.concat([feature_df, pd.DataFrame(embed_col.detach().numpy())], axis=1)

        return feature_df

In [54]:
t_user_cate_cols = user_cate_cols
t_feed_cate_cols = feed_cate_cols
t_user_num_cols = user_num_cols
t_feed_num_cols = feed_num_cols

In [55]:
t_usr_cat_df = train_dataset_builder_[t_user_cate_cols]
t_usr_num_df = train_dataset_builder_[t_user_num_cols]
t_item_cat_df = train_dataset_builder_[t_feed_cate_cols]
t_item_num_df = train_dataset_builder_[t_feed_num_cols]

t_usr_cat_df_prep = cat_embeding_prepare(t_usr_cat_df)
t_item_cat_df_prep = cat_embeding_prepare(t_item_cat_df)
t_usr_df = pd.concat([t_usr_cat_df_prep, t_usr_num_df], axis=1)
t_item_df = pd.concat([t_item_cat_df_prep, t_item_num_df], axis=1)

/usr/local/lib/python3.7/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/site-packages/pandas/core/frame.py:2957: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.loc._setitem_with_indexer(indexer, value)
/usr/local/li

In [56]:
print(t_usr_df.shape)
print(t_item_df.shape)

(100000, 815)
(100000, 212)
